# OCR_detected_license_plate_using_Tensorflow
## Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import random

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pytesseract

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

## Env setup

In [2]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [3]:
def yo_make_the_conversion(img_data, count):
    img = img_data
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    gray = cv2.medianBlur(gray, 3)
    path_png='png_tesseract'
    count += 1
    filename = os.path.join(path_png,'image{}.png'.format(count))
    cv2.imwrite(filename, gray)
    return filename

In [4]:
def check_array(tex):
    side=tex
    az='ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890'
    i=0
    yo=len(az)
    txt=''
    for i in range(0,yo):
        if side==az[i]:
            txt=az[i]
            break
    return txt   

In [5]:
def catch_rectify_plate_characters(text):
    tex = text
    out1=[]
    size=len(tex)
    for i in range(0,size):
      if tex[i]==check_array(tex[i]):
        out1.append(tex[i])
    yup=''.join(str(e) for e in out1)    
    return yup

In [6]:
MODEL_NAME = 'number_plate'
PATH_TO_CKPT = MODEL_NAME + '/graph-200000/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')
NUM_CLASSES = 1

### Load a (frozen) Tensorflow model into memory.

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.compat.v1.import_graph_def(od_graph_def, name='')

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Detection

In [9]:
from PIL import Image

In [10]:
PATH_TO_TEST_IMAGES_DIR = 'png_tesseract/test_ram'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.JPEG'.format(i)) for i in range(0,17) ]
IMAGE_SIZE = (12, 8)
TEST_DHARUN=os.path.join('number_plate')
count = 0

In [11]:
with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path) 
      image_np = cv2.imread(image_path,1)
      image_np_expanded = np.expand_dims(image_np, axis=0)
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      ymin = boxes[0,0,0]
      xmin = boxes[0,0,1]
      ymax = boxes[0,0,2]
      xmax = boxes[0,0,3]
      (im_width, im_height) = image.size
      (xminn, xmaxx, yminn, ymaxx) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
      cropped_image = tf.image.crop_to_bounding_box(image_np, int(yminn), int(xminn),int(ymaxx - yminn), int(xmaxx - xminn))
      img_data = sess.run(cropped_image)
      count = 0
      
      config = ('-l eng --oem 1 --psm 3')
      filename = yo_make_the_conversion(img_data, count)

      pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR'
      pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
      text = pytesseract.image_to_string(img_data,config=config)
      print('CHARCTER RECOGNITION : ',catch_rectify_plate_characters(text))
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=5)
      
      
      plt.figure(figsize=IMAGE_SIZE)
      plt.subplot(1,2,1)
      plt.imshow(image)
      plt.subplot(1,2,2)
      plt.imshow(img_data)
      plt.show()   


CHARCTER RECOGNITION :  HR696969


<ipython-input-11-1ad6773a6031>:48: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


CHARCTER RECOGNITION :  TN07BU5427
CHARCTER RECOGNITION :  HR26DK0830
CHARCTER RECOGNITION :  DL7CN5617
CHARCTER RECOGNITION :  DL3CAY9324
CHARCTER RECOGNITION :  
CHARCTER RECOGNITION :  GJW115A1138
CHARCTER RECOGNITION :  DL12CG6648
CHARCTER RECOGNITION :  UP14CP6748
CHARCTER RECOGNITION :  HR26BR9044
CHARCTER RECOGNITION :  HH20CS9817
CHARCTER RECOGNITION :  TN38MG6162
CHARCTER RECOGNITION :  TN38BY4191
CHARCTER RECOGNITION :  DL3CAY2231
CHARCTER RECOGNITION :  DL3CBF3907
CHARCTER RECOGNITION :  
CHARCTER RECOGNITION :  


In [12]:
def temp(image_path):
    with detection_graph.as_default():
          with tf.compat.v1.Session(graph=detection_graph) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')


            image = Image.open(image_path) 
            image_np = cv2.imread(image_path,1)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            ymin = boxes[0,0,0]
            xmin = boxes[0,0,1]
            ymax = boxes[0,0,2]
            xmax = boxes[0,0,3]
            (im_width, im_height) = image.size
            (xminn, xmaxx, yminn, ymaxx) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
            cropped_image = tf.image.crop_to_bounding_box(image_np, int(yminn), int(xminn),int(ymaxx - yminn), int(xmaxx - xminn))
            img_data = sess.run(cropped_image)
            count = 0

            config = ('-l eng --oem 1 --psm 3')
            filename = yo_make_the_conversion(img_data, count)

            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR'
            pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
            text = pytesseract.image_to_string(img_data,config=config)
            
            result = catch_rectify_plate_characters(text)
#             messagebox.showinfo('CHARCTER RECOGNITION : ',result)
            return (result,filename)

            


In [16]:
from tkinter import Tk,Button,Text,Canvas,PhotoImage,Label
from PIL import ImageTk, Image
from tkinter import messagebox
import requests

URL = "http://localhost:5000/list"



tkWindow = Tk()  
tkWindow.geometry('400x150')  
tkWindow.title('PythonExamples.org - Tkinter Example')
tkWindow.configure(background='lightblue')

def showMsg():  
    tkResult = Tk()  
    tkResult.geometry('400x150')  
    tkResult.title('Details of the Detected License Plate')
    tkResult.configure(background='lightgreen')
    var = inputtxt.get(1.0, "end-1c")
    result,filename = temp(var)
    Label(tkResult, text="CHARCTER RECOGNITION OUTPUT", font=('Caveat 15 bold')).pack(pady=20)
    Label(tkResult, text=result, font=('Caveat 15 bold')).pack(pady=20)
    PARAMS = {'x':result}
    r = requests.get(url = URL, params = PARAMS)
    print(r.text)
    
def showInputImg():
#     path = "png_tesseract/test_ram/image0.JPEG"
    pathToInput = inputtxt.get(1.0, "end-1c")
    img= Image.open(pathToInput)
    img=ImageTk.PhotoImage(img)
    label= Label(tkWindow, image= img)
    label.image= img
    label.pack()
    

# png_tesseract/test_ram/image0.JPEG

inputtxt = Text(tkWindow, height = 5, width = 20)
inputtxt.pack()

button = Button(tkWindow,
text = 'Show uploaded Image',
command = showInputImg)
button.pack()  

button = Button(tkWindow,
text = 'Submit',
command = showMsg)  
button.pack()  

 
  
tkWindow.mainloop()

In [18]:
URL = "http://localhost:5000/list"
PARAMS = {'x':"result"}
r = requests.get(url = URL, params = PARAMS)

print(r.text)

not found


In [122]:
from PIL import Image
PATH_TO_TEST_IMAGES_DIR = 'png_tesseract/test_ram'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.JPEG'.format(i)) for i in range(0,5) ]
print(TEST_IMAGE_PATHS[0])
temp("png_tesseract/test_ram/image0.JPEG")

png_tesseract/test_ram\image0.JPEG


('HR696969',
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=551x455 at 0x153D77B61C0>)

In [124]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from PIL import Image, ImageTk

URL = "http://localhost:5000/list"


#Create an instance of Tkinter frame
win= Tk()
win.configure(background='pink')

def showMsg():  
    var = inputtxt.get(1.0, "end-1c")
    result = temp(var)
    PARAMS = {'x':result}
    r = requests.get(url = URL, params = PARAMS)
    print(r.text)
    
    
#Define the geometry
win.geometry("750x350")
win.title("Image Gallery")
def select_file():
   path= filedialog.askopenfilename(title="Select an Image", filetype=(('image    files','*.jpeg'),('all files','*.*')))
   img= Image.open(path)
   img=ImageTk.PhotoImage(img)
   label= Label(win, image= img)
   label.image= img
   label.pack()
#Create a label and a Button to Open the dialog
Label(win, text="Click the Button below to select an Image", font=('Caveat 15 bold')).pack(pady=20)
button= ttk.Button(win, text="Select to Open", command= select_file)
button.pack(ipadx=5, pady=15)
button= ttk.Button(win, text="Get License Plate and Other Details", command= showMsg)
button.pack(ipadx=5, pady=15)
win.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Acer\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-124-69f03e8b06cb>", line 14, in showMsg
    var = inputtxt.get(1.0, "end-1c")
  File "C:\Users\Acer\anaconda3\lib\tkinter\__init__.py", line 3711, in get
    return self.tk.call(self._w, 'get', index1, index2)
_tkinter.TclError: invalid command name ".!text"


In [ ]:
from tkinter import Tk,Button,Text,Canvas,PhotoImage,Label
from PIL import ImageTk, Image
from tkinter import messagebox
import requests

URL = "http://localhost:5000/list"



tkWindow = Tk()  
tkWindow.geometry('400x150')  
tkWindow.title('PythonExamples.org - Tkinter Example')
tkWindow.configure(background='pink')

def showMsg():  
    var = inputtxt.get(1.0, "end-1c")
    result = temp(var)
    PARAMS = {'x':result}
    r = requests.get(url = URL, params = PARAMS)
    print(r.text)
    
def showInputImg():
#     path = "png_tesseract/test_ram/image0.JPEG"
    pathToInput = inputtxt.get(1.0, "end-1c")
    img= Image.open(pathToInput)
    img=ImageTk.PhotoImage(img)
    label= Label(win, image= img)
    

# png_tesseract/test_ram/image0.JPEG

inputtxt = Text(tkWindow, height = 5, width = 20)
inputtxt.pack()

button = Button(tkWindow,
text = 'Show uploaded Image',
command = showInputImg)
button.pack()  

button = Button(tkWindow,
text = 'Submit',
command = showMsg)  
button.pack()  

 
  
tkWindow.mainloop()

In [15]:
from tkinter import *



root = Tk()


class PokemonClass(object):

    def __init__(self, master):
        frame = Frame(master)
        frame.pack()

        self.WelcomeLabel = Label(root, text = "Welcome! Pick your Pokemon!", bg = "Black", fg = "White")
        self.WelcomeLabel.pack(fill = X)

        self.CharButton = Button(root, text = "Charmander", bg = "RED", fg = "White", command = CharClick)
        self.CharButton.pack(side = LEFT, fill = X)

        self.SquirtButton = Button(root, text = "Squirtle", bg = "Blue", fg = "White")
        self.SquirtButton.pack(side = LEFT, fill = X)

        self.BulbButton = Button(root, text = "Bulbasaur", bg = "Dark Green", fg = "White")
        self.BulbButton.pack(side = LEFT, fill = X)

    def CharClick():
        print ("You like Charmander!")
        CharPhoto = PhotoImage(file = "Charmander.gif")
        ChLabel = Label(root, image = CharPhoto)
        ChLabel.pack()


k = PokemonClass(root)
root.mainloop()